# SLCosmo Round-trip Test

In this notebook we exercise `SLCosmo`'s mock data generation and inference methods to simulate a TDC2 submission and infer a Hubble constant that we can compare with the input value. This is a functional test, in the sense that if our round trip does not get us back to where we started, there must be at least one bug in either or both of the mathematics or code. 


## Generating a Mock TDC2 Submission

First we do this.

## The TDC2 Probabilistic Model

As derived in the draft TDC2 design paper.

## Inference of $H_0$

Now we do this.

## Conclusions

* Bulleted lists are best for conclusions.